# Model
- LightGBM 사용
- 일반적 성능도 좋지만 속도가 빨라서 여러번 실험을 해볼 수 있었음
- 데이터의 크기가 너무 커서 파라미터 튜닝이 어려웠음
    - 직접 파라미터를 찍어가면서 validation이 좋은 단일 모델을 찾음  
- 단일 모델에 bagging 을 적용하여 30개의 모델을 평균으로 앙상블함

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [ ]:
# 재현을 위해 랜덤시드를 고정해 놓고 돌렸어야 했는데 실수로 시드 고정을 빠트림
# np.random.seed(12)

In [2]:
# 저장된 데이터 불러오기
x_train = pd.read_pickle('./data/x_train_1.pickle')
x_valid = pd.read_pickle('./data/x_val_1.pickle')
x_test = pd.read_pickle('./data/x_test_1.pickle')
y_train = pd.read_pickle('./data/Y_train_1.pickle')
y_valid = pd.read_pickle('./data/Y_valid_1.pickle')

In [5]:
params = {
    'learning_rate' : 0.11,
    'num_leaves' : 400,
    'n_estimators' : 300,
    'max_depth': -1,
    'min_child_weight' : 3, 
    'subsample' : 0.8,
    'colsample_bytree' : 0.5,
    'n_jobs': -1,
    'lambda' : 0.7
}

In [7]:
bagging_predict = [] # 예측 값 array 저장을 위한 리스트
for _ in range(30): # iteration 30회 - 30회는 임의로 정한 숫자
    data_index = [idx  for idx in range(x_train.shape[0])] # 전체 데이터의 인덱스를 추출한 리스트
    random_index = np.random.choice(data_index, x_train.shape[0]) # 복원추출 적용
    lgb_data = lgb.Dataset(data=x_train.iloc[random_index,], label=y_train.iloc[random_index,]) #  train dataset
    lgb_val = lgb.Dataset(data=x_valid, label=y_valid) # validation dataset
    lgb_model = lgb.train(params=params, train_set=lgb_data, valid_sets= lgb_val) # model train
    predict = lgb_model.predict(x_test) # prediction
    bagging_predict.append(predict) # 예측 값 array 저장

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [8]:
len(bagging_predict)

30

In [9]:
prediction = [] # 예측 값 리스트
for idx2 in range(x_test.shape[0]): # test의 index 만큼 iteration
    temp = []
    # 각 array line by line 으로 평균 내어 prediction에 저장
    for idx in range(len(bagging_predict)): 
        temp.append(bagging_predict[idx][idx2])
    prediction.append(np.mean(temp))

In [10]:
len(prediction)

214200

In [11]:
# test set
test=pd.read_csv("./data/test.csv")

In [15]:
# submission 파일 만들기
Y_test = np.array(prediction).clip(0,20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('lgb_submission3_en.csv', index=False)